In [1]:
"""Households

Demonstration of how to manage the holdings for an investor based on each Mandate & Household they are associated with.

Attributes
----------
instruments
portfolios
properties
set holdings
quotes
portfolio groups
aggregation
"""

'Households\n\nDemonstration of how to manage the holdings for an investor based on each Mandate & Household they are associated with.\n\nAttributes\n----------\ninstruments\nportfolios\nproperties\nset holdings\nquotes\nportfolio groups\naggregation\n'

## The Challenge

As a wealth manager you have a number of investors who are associated with multiple investment Mandates. Each Mandate may have one or more Accounts. These investors can be part of one or more Households, and a Household can contain one or more Investors. You would like to be able to see the holdings for an investor based on each Mandate & Household they are associated with. 

You also have a number of Branches and would like to see the assets managed by each Branch. Furthermore each branch has a number of Financial Advisors and you would like to be able to see the assets managed by each Financial Advisor.

## The Solution

1) Create your Instrument universe using a range of identifiers

2) Set up a scope for each branch to hold the accounts

3) Create a LUSID Portfolio for each account

4) Create a Property to hold the investor, mandate, and household details

5) Set the initial holdings of the portfolio

6) Load market data (prices)

7) Create a Portfolio Group for each household

8) Conduct a valuation against each household

9) Consider other ways of grouping & valuing client accounts

In [2]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import uuid
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.6.4966.0


![Scopes](img/paper-lusid.gif)

### Create your instrument universe using a range of identifiers

Before you can take on any holdings for your client accounts you need to ensure that your instrument universe has been populated. In this case you will import your instrument universe from a CSV file. Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument).

*Run the cell below to import your instrument universe*

In [3]:
equity_instruments = pd.read_csv('data/households-instruments-equities.csv')
equity_instruments.head(n=20)

InstrumentName ClientInternal Currency  \
0                         Amazon_Nasdaq_AMZN   imd_34634534      USD   
1                          Apple_Nasdaq_AAPL   imd_35345345      USD   
2               VANGUARD LIFESTRATEGY GROWTH   imd_89881022      USD   
3                         Salesforce.com Inc   imd_44953022      USD   
4         Norwegian Cruise Line Holdings Ltd   imd_44569345      USD   
5                           CAMPBELL SOUP CO   imd_44645943      USD   
6                           Under Armour Inc   imd_44678322      USD   
7                             Halliburton Co   imd_44588822      USD   
8                                   MSCI Inc   imd_44999201      USD   
9                          Brown-Forman Corp   imd_45992929      USD   
10            Blackrock Equity Dividend Fund   imd_49302011      USD   
11  T. Rowe Price Instl Small-Cap Stock Fund   imd_44904222      USD   

      QuotePermId ExchangeCode           CountryIssue Ticker MarketSector  \
0    1-8590928320           UN  united_states_america   AMZN       equity   
1    1-8590932301           UN  united_states_america   AAPL       equity   
2    1-5036190193           US  united_states_america  VASGX       equity   
3    1-8590926819           US  united_states_america    CRM       equity   
4   1-21475872704           US  united_states_america   NCLH       equity   
5    1-8590938673           US  united_states_america    CPB       equity   
6    1-8590940860           US  united_states_america     UA       equity   
7    1-8590930194           US  united_states_america    HAL       equity   
8    1-8590932969           US  united_states_america   MSCI       equity   
9    1-8590941544           US  united_states_america   BF/A       equity   
10   1-4297570798           US  united_states_america  MDDVX       equity   
11   1-4295989835           US  united_states_america  TRSSX       equity   

     SecurityType  Coupon  
0    common_stock     nan  
1    common_stock     nan  
2   fund_of_funds     nan  
3    common_stock     nan  
4    common_stock     nan  
5    common_stock     nan  
6    common_stock     nan  
7    common_stock     nan  
8    common_stock     nan  
9    common_stock     nan  
10  open_end_fund     nan  
11  open_end_fund     nan

Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument).

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [4]:
# Initialise your batch upsert request
batch_upsert_request = {}

# Using your instrument universe create your batch request
for index, instrument in equity_instruments.iterrows():

    # Specify the columns of your identifiers
    identifier_columns = ['QuotePermId', 'Ticker', 'ClientInternal']
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier] = models.InstrumentIdValue(
            value=instrument[identifier])
    
    # Build your request and add it to the dictionary
    batch_upsert_request[instrument['InstrumentName']] = models.InstrumentDefinition(
        name=instrument['InstrumentName'],
        identifiers=identifiers)
    
# Call LUSID to upsert your instrument definitions
instrument_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(request_body=batch_upsert_request)

# Pretty print the response
lusid_response_to_data_frame(list(instrument_response.values.values()))

href lusid_instrument_id  \
0   https://fbn-prd.lusid.com/api/api/instruments/...       LUID_1L72D4E9   
1   https://fbn-prd.lusid.com/api/api/instruments/...       LUID_UEP0W6G4   
2   https://fbn-prd.lusid.com/api/api/instruments/...       LUID_NT3GZV5U   
3   https://fbn-prd.lusid.com/api/api/instruments/...       LUID_27YIY39Z   
4   https://fbn-prd.lusid.com/api/api/instruments/...       LUID_OZ0A78EF   
5   https://fbn-prd.lusid.com/api/api/instruments/...       LUID_C5RGDBIJ   
6   https://fbn-prd.lusid.com/api/api/instruments/...       LUID_M3VJB8JN   
7   https://fbn-prd.lusid.com/api/api/instruments/...       LUID_KWWZD1OC   
8   https://fbn-prd.lusid.com/api/api/instruments/...       LUID_ZLPZOB53   
9   https://fbn-prd.lusid.com/api/api/instruments/...       LUID_WW73RZIP   
10  https://fbn-prd.lusid.com/api/api/instruments/...       LUID_W1UBLAHK   
11  https://fbn-prd.lusid.com/api/api/instruments/...       LUID_QUTQJ6OO   

       version.effective_from               version.as_at_date  \
0   0001-01-01 00:00:00+00:00 2020-08-06 06:53:53.378246+00:00   
1   0001-01-01 00:00:00+00:00 2020-08-06 06:53:53.378246+00:00   
2   0001-01-01 00:00:00+00:00 2020-08-06 06:53:53.378246+00:00   
3   0001-01-01 00:00:00+00:00 2020-08-06 06:53:53.378246+00:00   
4   0001-01-01 00:00:00+00:00 2020-08-06 06:53:53.378246+00:00   
5   0001-01-01 00:00:00+00:00 2020-08-06 06:53:53.378246+00:00   
6   0001-01-01 00:00:00+00:00 2020-08-06 06:53:53.378246+00:00   
7   0001-01-01 00:00:00+00:00 2020-08-06 06:53:53.378246+00:00   
8   0001-01-01 00:00:00+00:00 2020-08-06 06:53:53.378246+00:00   
9   0001-01-01 00:00:00+00:00 2020-08-06 06:53:53.378246+00:00   
10  0001-01-01 00:00:00+00:00 2020-08-06 06:53:53.378246+00:00   
11  0001-01-01 00:00:00+00:00 2020-08-06 06:53:53.378246+00:00   

                                        name identifiers.QuotePermId  \
0             Blackrock Equity Dividend Fund            1-4297570798   
1               VANGUARD LIFESTRATEGY GROWTH            1-5036190193   
2                           CAMPBELL SOUP CO            1-8590938673   
3                             Halliburton Co            1-8590930194   
4                                   MSCI Inc            1-8590932969   
5   T. Rowe Price Instl Small-Cap Stock Fund            1-4295989835   
6                          Brown-Forman Corp            1-8590941544   
7                         Amazon_Nasdaq_AMZN            1-8590928320   
8                           Under Armour Inc            1-8590940860   
9                         Salesforce.com Inc            1-8590926819   
10                         Apple_Nasdaq_AAPL            1-8590932301   
11        Norwegian Cruise Line Holdings Ltd           1-21475872704   

   identifiers.Ticker identifiers.LusidInstrumentId  \
0               MDDVX                 LUID_1L72D4E9   
1               VASGX                 LUID_UEP0W6G4   
2                 CPB                 LUID_NT3GZV5U   
3                 HAL                 LUID_27YIY39Z   
4                MSCI                 LUID_OZ0A78EF   
5               TRSSX                 LUID_C5RGDBIJ   
6                BF/A                 LUID_M3VJB8JN   
7                AMZN                 LUID_KWWZD1OC   
8                  UA                 LUID_ZLPZOB53   
9                 CRM                 LUID_WW73RZIP   
10               AAPL                 LUID_W1UBLAHK   
11               NCLH                 LUID_QUTQJ6OO   

   identifiers.ClientInternal properties   state identifiers.Isin  \
0                imd_49302011         []  Active              NaN   
1                imd_89881022         []  Active              NaN   
2                imd_44645943         []  Active              NaN   
3                imd_44588822         []  Active              NaN   
4                imd_44999201         []  Active              NaN   
5                imd_44904222         []  Active              NaN   
6                imd_45992929         []  Active    

![Scopes](img/households-instrumentmaster.gif)

### Set up a scope to hold the accounts

Your wealth managment company has a number of different branches. Each of these branches will be allocated a Scope in LUSID for its accounts. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

*Run the cell below to create a name for the scope of two of your branches*

In [5]:
# Get the ids for the scopes
scope_ids = [import_data.create_scope_id() for i in range(0,3)]

# Set up a scope for each branch
branch_singapore_scope = 'branch_{}'.format(scope_ids[0])
branch_hongkong_scope = 'branch_{}'.format(scope_ids[1])
branch_scopes = [branch_singapore_scope, branch_hongkong_scope]

# Set up a scope for reporting
reporting_scope = 'reporting_{}'.format(scope_ids[2])

# Pretty print the responses
prettyprint.heading('Singapore Branch Scope', branch_singapore_scope)
prettyprint.heading('Hong Kong Branch Scope', branch_hongkong_scope)
prettyprint.heading('Reporting Scope', reporting_scope)

Singapore Branch Scope: branch_38b9-df83-d0e9-6c
Hong Kong Branch Scope: branch_38b9-df83-d0e9-b0
Reporting Scope: reporting_38b9-df83-d0e9-c8


![Scopes](img/households-branch.gif)

### Create a portfolio for each account that exists with the branch

Now that you have decided on the name for your scope, you can create the portfolios to represent the client accounts inside this scope. You will import the account details from a CSV file.

*Run the cell below to import your client's account details*

In [6]:
# Import the account details
accounts = pd.read_csv('data/households-accounts.csv')
accounts.head(n=10)

account_code                   account_name currency  \
0  040004-929987648               OneIntlBrokerage      USD   
1  040004-778939522               OneIntlBrokerage      USD   
2  204507-250099511                        VangETF      USD   
3  774001-899930233  FidelityOnlineUSEquityOptions      USD   
4  400250-613060229          ActiveManagedBlckrock      USD   
5  769231-448679901                     MutualFund      USD   

                                       description primary_acount_owner_id  \
0                          Primary trading account        invstr_7325jhv93   
1                                    College fund         invstr_7325jhv93   
2                     Passive exchange traded fund        invstr_a1kf37761   
3                 Online US equity & option trades        invstr_bbgj93921   
4  Actively managed high return investment account        invstr_a1kf37761   
5                        Pooled investment vehicle        invstr_7325jhv93   

     household_id other_account_owner_id financial_advisor_id  \
0  hhd_jxgru45055                    NaN      ffid_abkjas9932   
1  hhd_jxgru45055       invstr_bbgj93921      ffid_jbogjh7878   
2   hhd_ab3452342                    NaN      ffid_jbogjh7878   
3  hhd_jxgru45055                    NaN      ffid_xxye990221   
4   hhd_ab3452342                    NaN      ffid_jbogjh7878   
5   hhd_ab3452342                    NaN      ffid_abkjas9932   

                                 mandate_description       mandate_id  \
0  To maximise returns within a given risk thresh...    mndt_9mdf9822   
1    To build a college fund for multiple dependants  mndt_99kljdslf0   
2  To hold 80% in low risk passive instruments an...   mndt_xxku9sdf9   
3  To maximise return with a high risk tolerance ...   mndt_uu99sadf2   
4  To hold 80% in low risk passive instruments an...   mndt_xxku9sdf9   
5  To maximise returns within a given risk thresh...    mndt_9mdf9822   

             branch_id  
0  Singapore_Singapore  
1  Singapore_Singapore  
2  Singapore_Singapore  
3         HongKong_SAR  
4  Singapore_Singapore  
5  Singapore_Singapore

With the account details loaded you can now create your client portfolios.

Note that every portfolio can be referenced by a unique code. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/what-is-2).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that when you create the portolios in the cell below you are creating it with a 'created' date of 1052 days ago. This number is rather arbitary, in practice it should be the date the portfolio came into existence regardless of the system you first created it in, read more about the importance of the created date on a portfolio in the [LUSID Knowledge Base: Importance of Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolios*

In [7]:
# A mapping between branch codes and scopes in LUSID
branch_mapping = {
    "Singapore_Singapore": branch_singapore_scope,
    "HongKong_SAR": branch_hongkong_scope
}

responses = []

# Iterate over the client accounts
for index, account in accounts.iterrows():

    # Set the creation date of your client portfolio 
    portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)
    
    # Build your request to create your client portfolio
    request = models.CreateTransactionPortfolioRequest(
        display_name=account['account_name'],
        code=account['account_code'],
        base_currency=account['currency'],
        description=account['description'],
        created=portfolio_creation_date,
        corporate_action_source_id=None,
        accounting_method='AverageCost',
        sub_holding_keys=None,
        properties=None)

    # Call LUSID to create your client portfolio
    responses.append(api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
        scope=branch_mapping[account['branch_id']],
        create_transaction_portfolio_request=request))

# Pretty print the response
lusid_response_to_data_frame(responses)

href  \
0  https://fbn-prd.lusid.com/api/api/portfolios/b...   
1  https://fbn-prd.lusid.com/api/api/portfolios/b...   
2  https://fbn-prd.lusid.com/api/api/portfolios/b...   
3  https://fbn-prd.lusid.com/api/api/portfolios/b...   
4  https://fbn-prd.lusid.com/api/api/portfolios/b...   
5  https://fbn-prd.lusid.com/api/api/portfolios/b...   

                   id.scope           id.code         type  \
0  branch_38b9-df83-d0e9-6c  040004-929987648  Transaction   
1  branch_38b9-df83-d0e9-6c  040004-778939522  Transaction   
2  branch_38b9-df83-d0e9-6c  204507-250099511  Transaction   
3  branch_38b9-df83-d0e9-b0  774001-899930233  Transaction   
4  branch_38b9-df83-d0e9-6c  400250-613060229  Transaction   
5  branch_38b9-df83-d0e9-6c  769231-448679901  Transaction   

                    display_name  \
0               OneIntlBrokerage   
1               OneIntlBrokerage   
2                        VangETF   
3  FidelityOnlineUSEquityOptions   
4          ActiveManagedBlckrock   
5                     MutualFund   

                                       description  \
0                          Primary trading account   
1                                    College fund    
2                     Passive exchange traded fund   
3                 Online US equity & option trades   
4  Actively managed high return investment account   
5                        Pooled investment vehicle   

                           created           version.effective_from  \
0 2017-09-19 06:53:53.638882+00:00 2017-09-19 06:53:53.638882+00:00   
1 2017-09-19 06:53:53.913214+00:00 2017-09-19 06:53:53.913214+00:00   
2 2017-09-19 06:53:54.212110+00:00 2017-09-19 06:53:54.212110+00:00   
3 2017-09-19 06:53:54.583875+00:00 2017-09-19 06:53:54.583875+00:00   
4 2017-09-19 06:53:54.863304+00:00 2017-09-19 06:53:54.863304+00:00   
5 2017-09-19 06:53:55.159412+00:00 2017-09-19 06:53:55.159412+00:00   

                version.as_at_date  is_derived  ...     links.5.relation  \
0 2020-08-06 06:53:53.832344+00:00       False  ...  HoldingsAdjustments   
1 2020-08-06 06:53:54.128457+00:00       False  ...  HoldingsAdjustments   
2 2020-08-06 06:53:54.453225+00:00       False  ...  HoldingsAdjustments   
3 2020-08-06 06:53:54.780665+00:00       False  ...  HoldingsAdjustments   
4 2020-08-06 06:53:55.066722+00:00       False  ...  HoldingsAdjustments   
5 2020-08-06 06:53:55.378432+00:00       False  ...  HoldingsAdjustments   

                                        links.5.href links.5.method  \
0  https://fbn-prd.lusid.com/api/api/transactionp...            GET   
1  https://fbn-prd.lusid.com/api/api/transactionp...            GET   
2  https://fbn-prd.lusid.com/api/api/transactionp...            GET   
3  https://fbn-prd.lusid.com/api/api/transactionp...            GET   
4  https://fbn-prd.lusid.com/api/api/transactionp...            GET   
5  https://fbn-prd.lusid.com/api/api/transactionp...            GET   

  links.6.relation                                       links.6.href  \
0     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   
1     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   
2     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   
3     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   
4     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   
5     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   

  links.6.method links.7.relation  \
0            GET      RequestLogs   
1            GET      RequestLogs   
2            GET      RequestLogs   
3            GET      RequestLogs   
4            GET      RequestLogs   
5            GET      RequestLogs   

                                        links.7.href  \
0  http://fbn-prd.lusid.com/app/insights/logs/0HM...   
1  http://fbn-prd.lusid.com/app/insights/logs/0HM...   
2  http://fbn-prd.lusid.com/app/insights/logs/0HM...   
3  http://fbn-prd.lusid.com/app/insights/logs/0HM...   
4  http

![Scopes](img/households-portfolios.gif)

### Create a property to hold the investor and the household

To keep track of the investor associated with each portfolio as well as the household you can make use of LUSID's extensible properties. These allow you to define a bespoke schema for your portfolio objects. Read more about properties in the [LUSID Knowledge Base: Properties](https://support.lusid.com/what-is-a-property). 

For further usage of the create property definition API call refer to the [LUSID API Docs: Create Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition).

*Run the cell below to create a property to hold household_id, the account owner ids & the financial advisor id*

In [8]:
# The property codes to create
property_codes = [
    'household_id', 
    'primary_acount_owner_id', 
    'other_account_owner_id',
    'financial_advisor_id',
    'mandate_id',
    'mandate_description',
    'branch_id'
]

responses = []

# Iterate over the property codes
for property_code in property_codes:

    # Create your request to define a new property
    request = models.CreatePropertyDefinitionRequest(
        domain='Portfolio',
        scope=reporting_scope,
        code=property_code,
        value_required=False,
        display_name=property_code,
        data_type_id=models.ResourceId(scope='system', code='string'))

    # Call LUSID to create your new property
    responses.append(api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
        create_property_definition_request=request))

# Pretty print the response
lusid_response_to_data_frame(responses)

key value_type  \
0  Portfolio/reporting_38b9-df83-d0e9-c8/househol...     String   
1  Portfolio/reporting_38b9-df83-d0e9-c8/primary_...     String   
2  Portfolio/reporting_38b9-df83-d0e9-c8/other_ac...     String   
3  Portfolio/reporting_38b9-df83-d0e9-c8/financia...     String   
4   Portfolio/reporting_38b9-df83-d0e9-c8/mandate_id     String   
5  Portfolio/reporting_38b9-df83-d0e9-c8/mandate_...     String   
6    Portfolio/reporting_38b9-df83-d0e9-c8/branch_id     String   

   value_required             display_name data_type_id.scope  \
0           False             household_id             system   
1           False  primary_acount_owner_id             system   
2           False   other_account_owner_id             system   
3           False     financial_advisor_id             system   
4           False               mandate_id             system   
5           False      mandate_description             system   
6           False                branch_id             system   

  data_type_id.code  life_time   type unit_schema     domain  \
0            string  Perpetual  Label     NoUnits  Portfolio   
1            string  Perpetual  Label     NoUnits  Portfolio   
2            string  Perpetual  Label     NoUnits  Portfolio   
3            string  Perpetual  Label     NoUnits  Portfolio   
4            string  Perpetual  Label     NoUnits  Portfolio   
5            string  Perpetual  Label     NoUnits  Portfolio   
6            string  Perpetual  Label     NoUnits  Portfolio   

                         scope                     code constraint_style  \
0  reporting_38b9-df83-d0e9-c8             household_id         Property   
1  reporting_38b9-df83-d0e9-c8  primary_acount_owner_id         Property   
2  reporting_38b9-df83-d0e9-c8   other_account_owner_id         Property   
3  reporting_38b9-df83-d0e9-c8     financial_advisor_id         Property   
4  reporting_38b9-df83-d0e9-c8               mandate_id         Property   
5  reporting_38b9-df83-d0e9-c8      mandate_description         Property   
6  reporting_38b9-df83-d0e9-c8                branch_id         Property   

  links.0.relation                                       links.0.href  \
0     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   
1     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   
2     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   
3     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   
4     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   
5     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   
6     EntitySchema  https://fbn-prd.lusid.com/api/api/schemas/enti...   

  links.0.method links.1.relation  \
0            GET      RequestLogs   
1            GET      RequestLogs   
2            GET      RequestLogs   
3            GET      RequestLogs   
4            GET      RequestLogs   
5            GET      RequestLogs   
6            GET      RequestLogs   

                                        links.1.href  \
0  http://fbn-prd.lusid.com/app/insights/logs/0HM...   
1  http://fbn-prd.lusid.com/app/insights/logs/0HM...   
2  http://fbn-prd.lusid.com/app/insights/logs/0HM...   
3  http://fbn-prd.lusid.com/app/insights/logs/0HM...   
4  http://fbn-prd.lusid.com/app/insights/logs/0HM...   
5  http://fbn-prd.lusid.com/app/insights/logs/0HM...   
6  http://fbn-prd.lusid.com/app/insights/logs/0HM...   

                                 links.1.description links.1.method  
0  A link to the LUSID Insights website showing a...            GET  
1  A link to the LUSID Insights website showing a...            GET  
2  A link to the LUSID Insights website showing a...            GET  
3  A link to the LUSID Insights website showing a...            GET  
4  A link to the LUSID Insights website showing a...            GET  
5  A link to the LUSID Insights website showing a...            GET  
6  A link to the LUSID Insights website showing 

You can also create a property which can hold multiple values. These are known as "Collection" properties.

*Run the cell below to create a collection property to hold all account owners*

In [9]:
# The collection property codes to create
property_codes_multi = {
    'account_owners': ['primary_acount_owner_id', 'other_account_owner_id']
}

responses = []

# Iterate over the property codes
for property_code in property_codes_multi.keys():

    # Create your request to define a new property
    request = models.CreatePropertyDefinitionRequest(
        domain='Portfolio',
        scope=reporting_scope,
        code=property_code,
        value_required=False,
        display_name=property_code,
        constraint_style="Collection",  # Note the constraint_style is set to "Collection"
        data_type_id=models.ResourceId(scope='system', code='string'))

    # Call LUSID to create your new property
    responses.append(api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
        create_property_definition_request=request))

# Pretty print the response
lusid_response_to_data_frame(responses)

key value_type  \
0  Portfolio/reporting_38b9-df83-d0e9-c8/account_...     String   

   value_required    display_name data_type_id.scope data_type_id.code  \
0           False  account_owners             system            string   

   life_time   type unit_schema     domain                        scope  \
0  Perpetual  Label     NoUnits  Portfolio  reporting_38b9-df83-d0e9-c8   

             code constraint_style links.0.relation  \
0  account_owners       Collection     EntitySchema   

                                        links.0.href links.0.method  \
0  https://fbn-prd.lusid.com/api/api/schemas/enti...            GET   

  links.1.relation                                       links.1.href  \
0      RequestLogs  http://fbn-prd.lusid.com/app/insights/logs/0HM...   

                                 links.1.description links.1.method  
0  A link to the LUSID Insights website showing a...            GET

With these properties created you can now populate them for each portfolio.

*Run the cell below to populate the values for these properties for each portfolio*

In [10]:
# Iterate over each account
for index, account in accounts.iterrows():
    
    # Add the relevant "Collection" account properties to the portfolio
    response = api_factory.build(lusid.api.PortfoliosApi).upsert_portfolio_properties(
        scope=branch_mapping[account['branch_id']],
        code=account['account_code'],
        request_body={
            'Portfolio/{}/{}'.format(reporting_scope, account_property): models.ModelProperty(
                key='Portfolio/{}/{}'.format(reporting_scope, account_property),
                value=models.PropertyValue(
                    label_value_set=models.LabelValueSet(values=[account[field] for field in fields if not pd.isnull(account[field])])
            )) for account_property, fields in property_codes_multi.items()
        }
    )
    
    # Pretty print the response 
    prettyprint.portfolio_properties_response(response)
    
    # Add the relevant Single Value account properties to the portfolio
    response = api_factory.build(lusid.api.PortfoliosApi).upsert_portfolio_properties(
        scope=branch_mapping[account['branch_id']],
        code=account['account_code'],
        request_body={
            'Portfolio/{}/{}'.format(reporting_scope, account_property): models.ModelProperty(
                key='Portfolio/{}/{}'.format(reporting_scope, account_property),
                value=models.PropertyValue(
                    label_value=account[account_property]
            )) for account_property in property_codes if not pd.isnull(account[account_property])
        }
    )

    # Pretty print the response 
    prettyprint.portfolio_properties_response(response)
    print ('\n')

Properties Sucessfully Updated for Portfolio
Property key: Portfolio/reporting_38b9-df83-d0e9-c8/account_owners
Values: ['invstr_7325jhv93']

Properties Sucessfully Updated for Portfolio
Property key: Portfolio/reporting_38b9-df83-d0e9-c8/household_id
Value: hhd_jxgru45055

Property key: Portfolio/reporting_38b9-df83-d0e9-c8/mandate_id
Value: mndt_9mdf9822

Property key: Portfolio/reporting_38b9-df83-d0e9-c8/mandate_description
Value: To maximise returns within a given risk threshold and focusing on investments with strong ESG scores

Property key: Portfolio/reporting_38b9-df83-d0e9-c8/primary_acount_owner_id
Value: invstr_7325jhv93

Property key: Portfolio/reporting_38b9-df83-d0e9-c8/branch_id
Value: Singapore_Singapore

Property key: Portfolio/reporting_38b9-df83-d0e9-c8/financial_advisor_id
Value: ffid_abkjas9932



Properties Sucessfully Updated for Portfolio
Property key: Portfolio/reporting_38b9-df83-d0e9-c8/account_owners
Values: ['invstr_bbgj93921', 'invstr_7325jhv93']

Propert

![Scopes](img/households-properties.gif)

### Set the initial holdings of the portfolio

Now that you have your instrument universe populated and portfolios created you can load your current client holdings into their portfolios. In this case you will import their holdings from a CSV file. 

*Run the cell below to import your current client holdings*

In [11]:
# Import and print the holdings
holdings = pd.read_csv('data/households-holdings.csv')
holdings.head(n=20)

portfolio_code                     instrument_name  quantity   price  \
0   040004-929987648                            USD_Cash     25000    1.00   
1   040004-929987648                  Amazon_Nasdaq_AMZN        31 1538.50   
2   040004-929987648                  Salesforce.com Inc        52  152.49   
3   040004-929987648  Norwegian Cruise Line Holdings Ltd        80   68.98   
4   040004-929987648                    CAMPBELL SOUP CO       150   44.25   
5   040004-929987648                    Under Armour Inc       300   22.52   
6   040004-929987648                   Apple_Nasdaq_AAPL        60  182.19   
7   040004-778939522                            USD_Cash     15000    1.00   
8   040004-778939522                   Brown-Forman Corp       250   59.86   
9   040004-778939522                      Halliburton Co        39   18.54   
10  040004-778939522                  Salesforce.com Inc       100  160.79   
11  040004-778939522                  Amazon_Nasdaq_AMZN         8 1487.92   
12  204507-250099511        VANGUARD LIFESTRATEGY GROWTH      3250   32.80   
13  204507-250099511                            USD_Cash      1523    1.00   
14  774001-899930233                    Under Armour Inc       300   22.99   
15  774001-899930233                      Halliburton Co       300   17.56   
16  774001-899930233                            MSCI Inc       155  234.50   
17  774001-899930233                            USD_Cash     25987    1.00   
18  400250-613060229      Blackrock Equity Dividend Fund       523   20.68   
19  400250-613060229                            USD_Cash      2800    1.00   

   currency    QuotePermId            branch_id  
0       USD            NaN  Singapore_Singapore  
1       USD   1-8590928320  Singapore_Singapore  
2       USD   1-8590926819  Singapore_Singapore  
3       USD  1-21475872704  Singapore_Singapore  
4       USD   1-8590938673  Singapore_Singapore  
5       USD   1-8590940860  Singapore_Singapore  
6       USD   1-8590932301  Singapore_Singapore  
7       USD            NaN  Singapore_Singapore  
8       USD   1-8590941544  Singapore_Singapore  
9       USD   1-8590930194  Singapore_Singapore  
10      USD   1-8590926819  Singapore_Singapore  
11      USD   1-8590928320  Singapore_Singapore  
12      USD   1-5036190193  Singapore_Singapore  
13      USD            NaN  Singapore_Singapore  
14      USD   1-8590940860         HongKong_SAR  
15      USD   1-8590930194         HongKong_SAR  
16      USD   1-8590932969         HongKong_SAR  
17      USD            NaN         HongKong_SAR  
18      USD   1-4297570798  Singapore_Singapore  
19      USD            NaN  Singapore_Singapore

Now that you have imported your client holdings you can add them to LUSID. You can do this by setting the holdings on a portfolio. Read more about how making an adjustment or setting the holdings on a portfolio affects it here [LUSID Knowledge Base: The effect of holding adjustments](https://support.lusid.com/how-do-holding-adjustments-affect-a-portfolio).

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to upsert your holdings into LUSID*

In [12]:
# Make the holdings effective from now
holdings_effective_date = datetime.now(pytz.UTC)

# Iterate over the portfolios in the holdings CSV
for portfolio in holdings['portfolio_code'].unique():
    
    # Initialise a list to hold your adjustments
    holding_adjustments = []
    
    # Iterate over the holdings in each portfolio
    for index, holding in holdings.loc[holdings['portfolio_code'] == portfolio].iterrows():
        
        # Set your instrument identifiers based on whether or not instrument is cash
        if 'Cash' in holding['instrument_name']:
            identifier_key = 'Instrument/default/Currency'
            identifer = holding['instrument_name'].split('_')[0]
        else:
            identifier_key = 'Instrument/default/QuotePermId'
            identifer = holding['QuotePermId']
            
        # Create your holding adjustment and append it to your list
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: identifer},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['quantity'] * holding['price'],
                            currency=holding['currency']),
                        portfolio_cost=holding['quantity'] * holding['price'],
                        price=holding['price'])
                ]
            )
        )
    
    # Call LUSID to set your initial holdings
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
        scope=branch_mapping[holding['branch_id']],
        code=portfolio,
        effective_at=holdings_effective_date,
        adjust_holding_request=holding_adjustments)

    # Pretty print our response from LUSID
    prettyprint.set_holdings_response(
        response, 
        branch_mapping[account['branch_id']], 
        portfolio)

Holdings Successfully Set for Portfolio
Scope: branch_38b9-df83-d0e9-6c
Code: 040004-929987648
Holdings Effective From: 2017-09-19 06:53:53.638882+00:00
Holdings Created On: 2020-08-06 06:54:08.572762+00:00

Holdings Successfully Set for Portfolio
Scope: branch_38b9-df83-d0e9-6c
Code: 040004-778939522
Holdings Effective From: 2017-09-19 06:53:53.913214+00:00
Holdings Created On: 2020-08-06 06:54:09.363541+00:00

Holdings Successfully Set for Portfolio
Scope: branch_38b9-df83-d0e9-6c
Code: 204507-250099511
Holdings Effective From: 2017-09-19 06:53:54.212110+00:00
Holdings Created On: 2020-08-06 06:54:10.039556+00:00

Holdings Successfully Set for Portfolio
Scope: branch_38b9-df83-d0e9-6c
Code: 774001-899930233
Holdings Effective From: 2017-09-19 06:53:54.583875+00:00
Holdings Created On: 2020-08-06 06:54:10.879539+00:00

Holdings Successfully Set for Portfolio
Scope: branch_38b9-df83-d0e9-6c
Code: 400250-613060229
Holdings Effective From: 2017-09-19 06:53:54.863304+00:00
Holdings Create

### Load market data prices

With the Portfolio Groups created to group each household together, you can now aggregate across households.

To aggregate & value a portfolio in LUSID you need to upsert market data quotes against the underlying holdings or specify an analytics library to use. Read more about aggregating and valuing portfolios in the [LUSID Knowledge Base: Aggregations and Valuations](https://support.lusid.com/what-is-a-valuation).

In this case you will upsert market data quotes to the quote store to be used in an aggregation request. You will import these quotes from a CSV file.

*Run the cell below to import the market data prices*

In [13]:
# Import the market data prices
prices = pd.read_csv('data/households-prices.csv')
prices.head(n=50)

instrument_name currency    QuotePermId  \
0               VANGUARD LIFESTRATEGY GROWTH      USD   1-5036190193   
1                         Amazon_Nasdaq_AMZN      USD   1-8590928320   
2                          Apple_Nasdaq_AAPL      USD   1-8590932301   
3                         Salesforce.com Inc      USD   1-8590926819   
4         Norwegian Cruise Line Holdings Ltd      USD  1-21475872704   
5                           CAMPBELL SOUP CO      USD   1-8590938673   
6                           Under Armour Inc      USD   1-8590940860   
7                             Halliburton Co      USD   1-8590930194   
8                                   MSCI Inc      USD   1-8590932969   
9                          Brown-Forman Corp      USD   1-8590941544   
10            Blackrock Equity Dividend Fund      USD   1-4297570798   
11  T. Rowe Price Instl Small-Cap Stock Fund      USD   1-4295989835   

   price_current ticker client_internal  
0           33.2  VASGX    imd_34634534  
1       1,738.50   AMZN    imd_35345345  
2         182.54   AAPL    imd_89881022  
3         158.44    CRM    imd_44953022  
4          52.38   NCLH    imd_44569345  
5          41.93    CPB    imd_44645943  
6          25.76     UA    imd_44678322  
7          21.19    HAL    imd_44588822  
8          234.5   MSCI    imd_44999201  
9           54.1   BF/A    imd_45992929  
10         20.72  MDDVX    imd_49302011  
11         24.19  TRSSX    imd_44904222

Now that you have imported the market data you can add it to the quote store in LUSID. Read more about what a quote is in the [LUSID Knowledge Base: What is a Quote?](https://support.lusid.com/what-is-a-quote).

For further usage of the Upsert Quotes API call refer to the [LUSID API Docs: Upsert Quotes](https://docs.lusid.com/#operation/UpsertQuotes).

*Run the cell below to upsert the market data quotes into LUSID*

In [14]:
# Initialise an empty list to hold the market data quotes
instrument_quotes = {}

# Iterate over each quote
for index, quote in prices.iterrows():
    
    # Get the LUSID Instrument ID for the quoted instrument
    luid = api_factory.build(lusid.api.SearchApi).instruments_search(
        instrument_search_property=[
            models.InstrumentSearchProperty(
                key='Instrument/default/QuotePermId',
                value=quote['QuotePermId'])
        ],
        mastered_only=True
        )[0].mastered_instruments[0].identifiers['LusidInstrumentId'].value
    
    # Create a quote for this instrument and append it to the list of quotes
    instrument_quotes[luid] = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider='DataScope',
                instrument_id=luid,
                instrument_id_type='LusidInstrumentId',
                quote_type='Price',
                field='Mid'),
            effective_at=holdings_effective_date
        ),
        metric_value=models.MetricValue(
            value=quote['price_current'],
            unit=quote['currency']),
        lineage='InternalSystem'
    )

# Upsert the quotes into LUSID
response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
    scope=reporting_scope,
    request_body=instrument_quotes)

# Pretty print the response
lusid_response_to_data_frame(list(response.values.values()))

quote_id.quote_series_id.provider quote_id.quote_series_id.instrument_id  \
0                          DataScope                          LUID_UEP0W6G4   
1                          DataScope                          LUID_KWWZD1OC   
2                          DataScope                          LUID_W1UBLAHK   
3                          DataScope                          LUID_WW73RZIP   
4                          DataScope                          LUID_QUTQJ6OO   
5                          DataScope                          LUID_NT3GZV5U   
6                          DataScope                          LUID_ZLPZOB53   
7                          DataScope                          LUID_27YIY39Z   
8                          DataScope                          LUID_OZ0A78EF   
9                          DataScope                          LUID_M3VJB8JN   
10                         DataScope                          LUID_1L72D4E9   
11                         DataScope                          LUID_C5RGDBIJ   

   quote_id.quote_series_id.instrument_id_type  \
0                            LusidInstrumentId   
1                            LusidInstrumentId   
2                            LusidInstrumentId   
3                            LusidInstrumentId   
4                            LusidInstrumentId   
5                            LusidInstrumentId   
6                            LusidInstrumentId   
7                            LusidInstrumentId   
8                            LusidInstrumentId   
9                            LusidInstrumentId   
10                           LusidInstrumentId   
11                           LusidInstrumentId   

   quote_id.quote_series_id.quote_type quote_id.quote_series_id.field  \
0                                Price                            Mid   
1                                Price                            Mid   
2                                Price                            Mid   
3                                Price                            Mid   
4                                Price                            Mid   
5                                Price                            Mid   
6                                Price                            Mid   
7                                Price                            Mid   
8                                Price                            Mid   
9                                Price                            Mid   
10                               Price                            Mid   
11                               Price                            Mid   

                quote_id.effective_at  metric_value.value metric_value.unit  \
0   2020-08-06T06:54:08.0694590+00:00               33.20               USD   
1   2020-08-06T06:54:08.0694590+00:00             1738.50               USD   
2   2020-08-06T06:54:08.0694590+00:00              182.54               USD   
3   2020-08-06T06:54:08.0694590+00:00              158.44               USD   
4   2020-08-06T06:54:08.0694590+00:00               52.38               USD   
5   2020-08-06T06:54:08.0694590+00:00               41.93               USD   
6   2020-08-06T06:54:08.0694590+00:00               25.76               USD   
7   2020-08-06T06:54:08.0694590+00:00               21.19               USD   
8   2020-08-06T06:54:08.0694590+00:00              234.50               USD   
9   2020-08-06T06:54:08.0694590+00:00               54.10               USD   
10  2020-08-06T06:54:08.0694590+00:00               20.72               USD   
11  2020-08-06T06:54:08.0694590+00:00               24.19               USD   

           lineage cut_label           uploaded_by  \
0   InternalSystem            00u4edwdnnCS5aFsN2p7   
1   InternalSystem            00u4edwdnnCS5aFsN2p7   
2   InternalSystem            00u4edwdnnCS5aFsN2p7   
3   InternalSystem            00u4edwdnnCS5aFsN2p7   
4   InternalSystem            00u4edwdnnCS5aFsN2p7   
5   InternalSystem    

### Create a portfolio group for & conduct a valuation against each household

Now that you've created Portfolios for the client accounts, you can group them together into households using Portfolio Groups.

Read more about portfolio groups here [LUSID Knowledge Base: How do you Group and Aggregate Portfolios?](https://support.lusid.com/how-do-you-group-and-aggregate-portfolios)

*Run the cell below to create a function which allows you to create the portfolio groups and add the relevant portfolios*

In [15]:
def create_portfolio_groups(account_data, grouping_key, group_scope, portfolio_scopes):
    """
    param: account_data (DataFrame) - The Pandas DataFrame with the account data
    param: grouping_key (str) - The key to group the accounts by 
    param: group_scope (str) - The scope to create the portfolio group in
    param: portfolio_scopes (list[str]) - The list of scopes that contain the 
    accounts to group
    
    returns: N/A
    """
    
    group_creation_date = datetime.now(pytz.UTC) - timedelta(days=5000)
    portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)
    
    # Raise an error saying that the key does not exist in the data
    if grouping_key not in account_data.columns:
        
        raise("""
            The grouping key does not exist in the account data! Please
            check your spelling and that the key exists in the data and 
            try again
            """)
        
    # Get all the unique groups from the grouping key
    for group in account_data[grouping_key].unique():
        
        # Build a create group request for this group
        group_request = models.CreatePortfolioGroupRequest(
            code=group+'-Group',
            display_name='Contains all accounts for the {} of {}'.format(
                grouping_key, group),
            created=group_creation_date)

        # Call LUSID to create the portfolio group
        response = api_factory.build(lusid.api.PortfolioGroupsApi).create_portfolio_group(
            scope=group_scope,
            create_portfolio_group_request=group_request)

        # Pretty print the response
        prettyprint.portfolio_group_response(response, 'created')
    
    # Initialise a list to hold all account portfolios
    portfolios = []

    # Iterate over all the scopes
    for portfolio_scope in portfolio_scopes:
        
        # Call LUSID to list all the portfolios across all the portfolio scopes
        response = api_factory.build(lusid.api.PortfoliosApi).list_portfolios_for_scope(
            scope=portfolio_scope)
        
        # Loop over each portfolio
        for portfolio in response.values:

            # For this portfolio get its properties which you defined earlier
            properties = api_factory.build(lusid.api.PortfoliosApi).get_portfolio_properties(
                scope=portfolio.id.scope, 
                code=portfolio.id.code)

            # Make the list of properties easy to work with by converting them to key-value pairs
            portfolio_properties = properties.properties

            # Using the group property determine which portfolio group to add this portfolio too
            response = api_factory.build(lusid.api.PortfolioGroupsApi).add_portfolio_to_group(
                scope=group_scope,
                code=portfolio_properties[
                    'Portfolio/{}/{}'.format(
                        group_scope, grouping_key)].value.label_value+'-Group',
                resource_id=models.ResourceId(
                    scope=portfolio.id.scope,
                    code=portfolio.id.code),
                effective_at=portfolio_creation_date)

            # Pretty print the response
            prettyprint.get_portfolio_group_response(response)

Now that you've defined your function you can use it to create portfolio groups for each household.

*Run the cell below to group the portfolios by household*

In [16]:
create_portfolio_groups(
    account_data=accounts, 
    grouping_key='household_id', 
    group_scope=reporting_scope, 
    portfolio_scopes=branch_scopes)

Portfolio Group Created
Name: Contains all accounts for the household_id of hhd_jxgru45055
Scope: reporting_38b9-df83-d0e9-c8
Code: hhd_jxgru45055-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the household_id of hhd_ab3452342
Scope: reporting_38b9-df83-d0e9-c8
Code: hhd_ab3452342-Group
Portfolios Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the household_id of hhd_jxgru45055
Scope: reporting_38b9-df83-d0e9-c8
Code: hhd_jxgru45055-Group
Portfolios Inside Group: 
040004-778939522
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the household_id of hhd_jxgru45055
Scope: reporting_38b9-df83-d0e9-c8
Code: hhd_jxgru45055-Group
Portfolios Inside Group: 
040004-778939522
040004-929987648
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the household_id of hhd_ab3452342
Scope: reporting_38b9-df83-d0e9-c8
Code: hhd_ab3452342-Group
Portfolios Inside Group: 
204507-250099511
S

Using these portfolio groups you can perform an aggregation across each household. The logic for an aggregation is controled by a LUSID recipe. Read more about recipes in the [LUSID Knowledge Base: What is a Recipe and How Are They Used?](https://support.lusid.com/what-is-a-recipe-and-how-are-they-used).

For further usage of the Get Aggregation by Portfolio API call refer to the [LUSID API Docs: Get Aggregation by Portfolio](https://docs.lusid.com/#operation/GetAggregationByPortfolio).

*Run the cell below to create a function to aggregate and value each household's investments*

In [17]:
def aggregate_portfolio_group(grouping_key, group_scope):
    """
    param: grouping_key (str) - The key to group the accounts by 
    param: group_scope (str) - The scope to the portfolio group is in
    
    returns: aggregation_results (list[DataFrame]) - The list of resulting
    aggregation dataframes
    """
    
    # Create an inline recipe to configue the aggregation logic
    inline_recipe = models.ConfigurationRecipe(
        scope="User",
        code='quotes_recipe',
        market=models.MarketContext(
            market_rules=[
                models.MarketDataKeyRule(
                   key='Equity.LusidInstrumentId.*',
                   supplier='DataScope',
                   data_scope=reporting_scope,
                   quote_type='Price',
                   field='Mid')
            ],
            suppliers=models.MarketContextSuppliers(
                commodity='DataScope',
                credit='DataScope',
                equity='DataScope',
                fx='DataScope',
                rates='DataScope'),
            options=models.MarketOptions(
                default_supplier='DataScope',
                default_instrument_code_type='LusidInstrumentId',
                default_scope=reporting_scope)
            )
        )
    
    # Initialse a list to hold the aggregation results
    aggregation_results_df = []

    # Iterate over each household
    for group in accounts[grouping_key].unique():

        # Create the aggregation request
        aggregation_request = models.AggregationRequest(
            inline_recipe=inline_recipe,
            effective_at=holdings_effective_date,
            metrics=[
                models.AggregateSpec(key='Holding/default/SubHoldingKey',
                op='Value'),
                models.AggregateSpec(key='Holding/default/Cost',
                op='Sum'),
                models.AggregateSpec(key='Holding/default/PV',
                op='Sum'),
                models.AggregateSpec(key='Holding/default/Units',
                op='Sum'),
                models.AggregateSpec(key='Instrument/default/Name',
                op='Value')
            ],
            group_by=[
                'Instrument/default/Name'
            ], 
            portfolio_identifier_code='GroupPortfolio') 

        # Call LUSID to perform an aggregation
        response = api_factory.build(lusid.api.AggregationApi).get_aggregation(
            scope=group_scope,
            code=group+'-Group',
            aggregation_request=aggregation_request)

        # Pretty print the response
        aggregation_results_df.append(prettyprint.aggregation_response_generic_df(
            response=response, 
            index_key="Instrument/default/Name", 
            name=group))
        
    return aggregation_results_df

Now that you've defined your function you can use it to aggregate across each household.

*Run the cell below to aggregate across each household*

In [18]:
aggregation_results_df = aggregate_portfolio_group(
    grouping_key='household_id', 
    group_scope=reporting_scope)

In [19]:
prettyprint.heading('Household', aggregation_results_df[0].columns.name)
aggregation_results_df[0]

Household: hhd_jxgru45055


hhd_jxgru45055                            Holding/default/SubHoldingKey  \
Instrument/default/Name                                                   
Amazon_Nasdaq_AMZN                  LusidInstrumentId=LUID_KWWZD1OC/USD   
Apple_Nasdaq_AAPL                   LusidInstrumentId=LUID_W1UBLAHK/USD   
Brown-Forman Corp                   LusidInstrumentId=LUID_M3VJB8JN/USD   
CAMPBELL SOUP CO                    LusidInstrumentId=LUID_NT3GZV5U/USD   
Halliburton Co                      LusidInstrumentId=LUID_27YIY39Z/USD   
MSCI Inc                            LusidInstrumentId=LUID_OZ0A78EF/USD   
Norwegian Cruise Line Holdings Ltd  LusidInstrumentId=LUID_QUTQJ6OO/USD   
Salesforce.com Inc                  LusidInstrumentId=LUID_WW73RZIP/USD   
USD                                                        Currency=USD   
Under Armour Inc                    LusidInstrumentId=LUID_ZLPZOB53/USD   
TOTAL                                                               NaN   

hhd_jxgru45055                      Sum(Holding/default/Cost)  \
Instrument/default/Name                                         
Amazon_Nasdaq_AMZN                                  59,596.86   
Apple_Nasdaq_AAPL                                    10,931.4   
Brown-Forman Corp                                    14,965.0   
CAMPBELL SOUP CO                                      6,637.5   
Halliburton Co                                       5,991.06   
MSCI Inc                                             36,347.5   
Norwegian Cruise Line Holdings Ltd                    5,518.4   
Salesforce.com Inc                                  24,008.48   
USD                                                  65,987.0   
Under Armour Inc                                     13,653.0   
TOTAL                                               243,636.2   

hhd_jxgru45055                      Sum(Holding/default/PV)  \
Instrument/default/Name                                       
Amazon_Nasdaq_AMZN                                 67,801.5   
Apple_Nasdaq_AAPL                                  10,952.4   
Brown-Forman Corp                                  13,525.0   
CAMPBELL SOUP CO                                    6,289.5   
Halliburton Co                                     7,183.41   
MSCI Inc                                           36,347.5   
Norwegian Cruise Line Holdings Ltd                  4,190.4   
Salesforce.com Inc                                24,082.88   
USD                                                65,987.0   
Under Armour Inc                                   15,456.0   
TOTAL                                            251,815.59   

hhd_jxgru45055                      Sum(Holding/default/Units)  
Instrument/default/Name                                         
Amazon_Nasdaq_AMZN                                        39.0  
Apple_Nasdaq_AAPL                                         60.0  
Brown-Forman Corp                                        250.0  
CAMPBELL SOUP CO                                         150.0  
Halliburton Co                                           339.0  
MSCI Inc                                                 155.0  
Norwegian Cruise Line Holdings Ltd                        80.0  
Salesforce.com Inc                                       152.0  
USD                                                   65,987.0  
Under Armour Inc                                         600.0  
TOTAL                                                 67,812.0

In [20]:
prettyprint.heading('Household', aggregation_results_df[1].columns.name)
aggregation_results_df[1]

Household: hhd_ab3452342


hhd_ab3452342                                   Holding/default/SubHoldingKey  \
Instrument/default/Name                                                         
Blackrock Equity Dividend Fund            LusidInstrumentId=LUID_1L72D4E9/USD   
T. Rowe Price Instl Small-Cap Stock Fund  LusidInstrumentId=LUID_C5RGDBIJ/USD   
USD                                                              Currency=USD   
VANGUARD LIFESTRATEGY GROWTH              LusidInstrumentId=LUID_UEP0W6G4/USD   
TOTAL                                                                     NaN   

hhd_ab3452342                             Sum(Holding/default/Cost)  \
Instrument/default/Name                                               
Blackrock Equity Dividend Fund                            10,815.64   
T. Rowe Price Instl Small-Cap Stock Fund                  13,555.28   
USD                                                         6,523.0   
VANGUARD LIFESTRATEGY GROWTH                              106,600.0   
TOTAL                                                    137,493.92   

hhd_ab3452342                             Sum(Holding/default/PV)  \
Instrument/default/Name                                             
Blackrock Equity Dividend Fund                          10,836.56   
T. Rowe Price Instl Small-Cap Stock Fund                13,449.64   
USD                                                       6,523.0   
VANGUARD LIFESTRATEGY GROWTH                            107,900.0   
TOTAL                                                   138,709.2   

hhd_ab3452342                             Sum(Holding/default/Units)  
Instrument/default/Name                                               
Blackrock Equity Dividend Fund                                 523.0  
T. Rowe Price Instl Small-Cap Stock Fund                       556.0  
USD                                                          6,523.0  
VANGUARD LIFESTRATEGY GROWTH                                 3,250.0  
TOTAL                                                       10,852.0

![Scopes](img/households-portfoliogroupshouseholds.gif)

### Group & aggregate by financial advisor

Perhaps you might also like to group your client accounts by financial advisor. Using your functions you defined earlier you can do this as well.

*Run the cell below to create groups for each advisor and perform aggregations*

In [21]:
# Create the portfolio groups for each financial advisor
create_portfolio_groups(
    account_data=accounts, 
    grouping_key='financial_advisor_id', 
    group_scope=reporting_scope, 
    portfolio_scopes=branch_scopes)

# Aggregate across the portfolio groups
aggregation_results_df = aggregate_portfolio_group(
    grouping_key='financial_advisor_id', 
    group_scope=reporting_scope)

Portfolio Group Created
Name: Contains all accounts for the financial_advisor_id of ffid_abkjas9932
Scope: reporting_38b9-df83-d0e9-c8
Code: ffid_abkjas9932-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the financial_advisor_id of ffid_jbogjh7878
Scope: reporting_38b9-df83-d0e9-c8
Code: ffid_jbogjh7878-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the financial_advisor_id of ffid_xxye990221
Scope: reporting_38b9-df83-d0e9-c8
Code: ffid_xxye990221-Group
Portfolios Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the financial_advisor_id of ffid_jbogjh7878
Scope: reporting_38b9-df83-d0e9-c8
Code: ffid_jbogjh7878-Group
Portfolios Inside Group: 
040004-778939522
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the financial_advisor_id of ffid_abkjas9932
Scope: reporting_38b9-df83-d0e9-c8
Code: ffid_abkjas9932-Group
Portfolios Inside Group: 
040004-929987648


In [22]:
prettyprint.heading('Financial Advisor', aggregation_results_df[0].columns.name)
aggregation_results_df[0]

Financial Advisor: ffid_abkjas9932


ffid_abkjas9932                                 Holding/default/SubHoldingKey  \
Instrument/default/Name                                                         
Amazon_Nasdaq_AMZN                        LusidInstrumentId=LUID_KWWZD1OC/USD   
Apple_Nasdaq_AAPL                         LusidInstrumentId=LUID_W1UBLAHK/USD   
CAMPBELL SOUP CO                          LusidInstrumentId=LUID_NT3GZV5U/USD   
Norwegian Cruise Line Holdings Ltd        LusidInstrumentId=LUID_QUTQJ6OO/USD   
Salesforce.com Inc                        LusidInstrumentId=LUID_WW73RZIP/USD   
T. Rowe Price Instl Small-Cap Stock Fund  LusidInstrumentId=LUID_C5RGDBIJ/USD   
USD                                                              Currency=USD   
Under Armour Inc                          LusidInstrumentId=LUID_ZLPZOB53/USD   
TOTAL                                                                     NaN   

ffid_abkjas9932                           Sum(Holding/default/Cost)  \
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN                                         47,693.5   
Apple_Nasdaq_AAPL                                          10,931.4   
CAMPBELL SOUP CO                                            6,637.5   
Norwegian Cruise Line Holdings Ltd                          5,518.4   
Salesforce.com Inc                                         7,929.48   
T. Rowe Price Instl Small-Cap Stock Fund                  13,555.28   
USD                                                        27,200.0   
Under Armour Inc                                            6,756.0   
TOTAL                                                    126,221.56   

ffid_abkjas9932                           Sum(Holding/default/PV)  \
Instrument/default/Name                                             
Amazon_Nasdaq_AMZN                                       53,893.5   
Apple_Nasdaq_AAPL                                        10,952.4   
CAMPBELL SOUP CO                                          6,289.5   
Norwegian Cruise Line Holdings Ltd                        4,190.4   
Salesforce.com Inc                                       8,238.88   
T. Rowe Price Instl Small-Cap Stock Fund                13,449.64   
USD                                                      27,200.0   
Under Armour Inc                                          7,728.0   
TOTAL                                                  131,942.32   

ffid_abkjas9932                           Sum(Holding/default/Units)  
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN                                              31.0  
Apple_Nasdaq_AAPL                                               60.0  
CAMPBELL SOUP CO                                               150.0  
Norwegian Cruise Line Holdings Ltd                              80.0  
Salesforce.com Inc                                              52.0  
T. Rowe Price Instl Small-Cap Stock Fund                       556.0  
USD                                                         27,200.0  
Under Armour Inc                                               300.0  
TOTAL                                                       28,429.0

In [23]:
prettyprint.heading('Financial Advisor', aggregation_results_df[1].columns.name)
aggregation_results_df[1]

Financial Advisor: ffid_jbogjh7878


ffid_jbogjh7878                       Holding/default/SubHoldingKey  \
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN              LusidInstrumentId=LUID_KWWZD1OC/USD   
Blackrock Equity Dividend Fund  LusidInstrumentId=LUID_1L72D4E9/USD   
Brown-Forman Corp               LusidInstrumentId=LUID_M3VJB8JN/USD   
Halliburton Co                  LusidInstrumentId=LUID_27YIY39Z/USD   
Salesforce.com Inc              LusidInstrumentId=LUID_WW73RZIP/USD   
USD                                                    Currency=USD   
VANGUARD LIFESTRATEGY GROWTH    LusidInstrumentId=LUID_UEP0W6G4/USD   
TOTAL                                                           NaN   

ffid_jbogjh7878                 Sum(Holding/default/Cost)  \
Instrument/default/Name                                     
Amazon_Nasdaq_AMZN                              11,903.36   
Blackrock Equity Dividend Fund                  10,815.64   
Brown-Forman Corp                                14,965.0   
Halliburton Co                                     723.06   
Salesforce.com Inc                               16,079.0   
USD                                              19,323.0   
VANGUARD LIFESTRATEGY GROWTH                    106,600.0   
TOTAL                                          180,409.06   

ffid_jbogjh7878                 Sum(Holding/default/PV)  \
Instrument/default/Name                                   
Amazon_Nasdaq_AMZN                             13,908.0   
Blackrock Equity Dividend Fund                10,836.56   
Brown-Forman Corp                              13,525.0   
Halliburton Co                                   826.41   
Salesforce.com Inc                             15,844.0   
USD                                            19,323.0   
VANGUARD LIFESTRATEGY GROWTH                  107,900.0   
TOTAL                                        182,162.97   

ffid_jbogjh7878                 Sum(Holding/default/Units)  
Instrument/default/Name                                     
Amazon_Nasdaq_AMZN                                     8.0  
Blackrock Equity Dividend Fund                       523.0  
Brown-Forman Corp                                    250.0  
Halliburton Co                                        39.0  
Salesforce.com Inc                                   100.0  
USD                                               19,323.0  
VANGUARD LIFESTRATEGY GROWTH                       3,250.0  
TOTAL                                             23,493.0

In [24]:
prettyprint.heading('Financial Advisor', aggregation_results_df[2].columns.name)
aggregation_results_df[2]

Financial Advisor: ffid_xxye990221


ffid_xxye990221                Holding/default/SubHoldingKey  \
Instrument/default/Name                                        
Halliburton Co           LusidInstrumentId=LUID_27YIY39Z/USD   
MSCI Inc                 LusidInstrumentId=LUID_OZ0A78EF/USD   
USD                                             Currency=USD   
Under Armour Inc         LusidInstrumentId=LUID_ZLPZOB53/USD   
TOTAL                                                    NaN   

ffid_xxye990221          Sum(Holding/default/Cost)  Sum(Holding/default/PV)  \
Instrument/default/Name                                                       
Halliburton Co                             5,268.0                  6,357.0   
MSCI Inc                                  36,347.5                 36,347.5   
USD                                       25,987.0                 25,987.0   
Under Armour Inc                           6,897.0                  7,728.0   
TOTAL                                     74,499.5                 76,419.5   

ffid_xxye990221          Sum(Holding/default/Units)  
Instrument/default/Name                              
Halliburton Co                                300.0  
MSCI Inc                                      155.0  
USD                                        25,987.0  
Under Armour Inc                              300.0  
TOTAL                                      26,742.0

![Scopes](img/households-portfoliogroupsadvisor.gif)

### Group & aggregate by investment mandate

You can even group and aggregate by investment mandate. 

*Run the cell below to group and aggregate by investment mandate*

In [25]:
# Create the portfolio groups for each mandate
create_portfolio_groups(
    account_data=accounts, 
    grouping_key='mandate_id', 
    group_scope=reporting_scope, 
    portfolio_scopes=branch_scopes)

# Aggregate across the portfolio groups
aggregation_results_df = aggregate_portfolio_group(
    grouping_key='mandate_id', 
    group_scope=reporting_scope)

Portfolio Group Created
Name: Contains all accounts for the mandate_id of mndt_9mdf9822
Scope: reporting_38b9-df83-d0e9-c8
Code: mndt_9mdf9822-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the mandate_id of mndt_99kljdslf0
Scope: reporting_38b9-df83-d0e9-c8
Code: mndt_99kljdslf0-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the mandate_id of mndt_xxku9sdf9
Scope: reporting_38b9-df83-d0e9-c8
Code: mndt_xxku9sdf9-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the mandate_id of mndt_uu99sadf2
Scope: reporting_38b9-df83-d0e9-c8
Code: mndt_uu99sadf2-Group
Portfolios Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the mandate_id of mndt_99kljdslf0
Scope: reporting_38b9-df83-d0e9-c8
Code: mndt_99kljdslf0-Group
Portfolios Inside Group: 
040004-778939522
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the mandate_id of

In [26]:
prettyprint.heading('Mandate', aggregation_results_df[0].columns.name)
aggregation_results_df[0]

Mandate: mndt_9mdf9822


mndt_9mdf9822                                   Holding/default/SubHoldingKey  \
Instrument/default/Name                                                         
Amazon_Nasdaq_AMZN                        LusidInstrumentId=LUID_KWWZD1OC/USD   
Apple_Nasdaq_AAPL                         LusidInstrumentId=LUID_W1UBLAHK/USD   
CAMPBELL SOUP CO                          LusidInstrumentId=LUID_NT3GZV5U/USD   
Norwegian Cruise Line Holdings Ltd        LusidInstrumentId=LUID_QUTQJ6OO/USD   
Salesforce.com Inc                        LusidInstrumentId=LUID_WW73RZIP/USD   
T. Rowe Price Instl Small-Cap Stock Fund  LusidInstrumentId=LUID_C5RGDBIJ/USD   
USD                                                              Currency=USD   
Under Armour Inc                          LusidInstrumentId=LUID_ZLPZOB53/USD   
TOTAL                                                                     NaN   

mndt_9mdf9822                             Sum(Holding/default/Cost)  \
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN                                         47,693.5   
Apple_Nasdaq_AAPL                                          10,931.4   
CAMPBELL SOUP CO                                            6,637.5   
Norwegian Cruise Line Holdings Ltd                          5,518.4   
Salesforce.com Inc                                         7,929.48   
T. Rowe Price Instl Small-Cap Stock Fund                  13,555.28   
USD                                                        27,200.0   
Under Armour Inc                                            6,756.0   
TOTAL                                                    126,221.56   

mndt_9mdf9822                             Sum(Holding/default/PV)  \
Instrument/default/Name                                             
Amazon_Nasdaq_AMZN                                       53,893.5   
Apple_Nasdaq_AAPL                                        10,952.4   
CAMPBELL SOUP CO                                          6,289.5   
Norwegian Cruise Line Holdings Ltd                        4,190.4   
Salesforce.com Inc                                       8,238.88   
T. Rowe Price Instl Small-Cap Stock Fund                13,449.64   
USD                                                      27,200.0   
Under Armour Inc                                          7,728.0   
TOTAL                                                  131,942.32   

mndt_9mdf9822                             Sum(Holding/default/Units)  
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN                                              31.0  
Apple_Nasdaq_AAPL                                               60.0  
CAMPBELL SOUP CO                                               150.0  
Norwegian Cruise Line Holdings Ltd                              80.0  
Salesforce.com Inc                                              52.0  
T. Rowe Price Instl Small-Cap Stock Fund                       556.0  
USD                                                         27,200.0  
Under Armour Inc                                               300.0  
TOTAL                                                       28,429.0

In [27]:
prettyprint.heading('Mandate', aggregation_results_df[1].columns.name)
aggregation_results_df[1]

Mandate: mndt_99kljdslf0


mndt_99kljdslf0                Holding/default/SubHoldingKey  \
Instrument/default/Name                                        
Amazon_Nasdaq_AMZN       LusidInstrumentId=LUID_KWWZD1OC/USD   
Brown-Forman Corp        LusidInstrumentId=LUID_M3VJB8JN/USD   
Halliburton Co           LusidInstrumentId=LUID_27YIY39Z/USD   
Salesforce.com Inc       LusidInstrumentId=LUID_WW73RZIP/USD   
USD                                             Currency=USD   
TOTAL                                                    NaN   

mndt_99kljdslf0          Sum(Holding/default/Cost)  Sum(Holding/default/PV)  \
Instrument/default/Name                                                       
Amazon_Nasdaq_AMZN                       11,903.36                 13,908.0   
Brown-Forman Corp                         14,965.0                 13,525.0   
Halliburton Co                              723.06                   826.41   
Salesforce.com Inc                        16,079.0                 15,844.0   
USD                                       15,000.0                 15,000.0   
TOTAL                                    58,670.42                59,103.41   

mndt_99kljdslf0          Sum(Holding/default/Units)  
Instrument/default/Name                              
Amazon_Nasdaq_AMZN                              8.0  
Brown-Forman Corp                             250.0  
Halliburton Co                                 39.0  
Salesforce.com Inc                            100.0  
USD                                        15,000.0  
TOTAL                                      15,397.0

In [28]:
prettyprint.heading('Mandate', aggregation_results_df[2].columns.name)
aggregation_results_df[2]

Mandate: mndt_xxku9sdf9


mndt_xxku9sdf9                        Holding/default/SubHoldingKey  \
Instrument/default/Name                                               
Blackrock Equity Dividend Fund  LusidInstrumentId=LUID_1L72D4E9/USD   
USD                                                    Currency=USD   
VANGUARD LIFESTRATEGY GROWTH    LusidInstrumentId=LUID_UEP0W6G4/USD   
TOTAL                                                           NaN   

mndt_xxku9sdf9                  Sum(Holding/default/Cost)  \
Instrument/default/Name                                     
Blackrock Equity Dividend Fund                  10,815.64   
USD                                               4,323.0   
VANGUARD LIFESTRATEGY GROWTH                    106,600.0   
TOTAL                                          121,738.64   

mndt_xxku9sdf9                  Sum(Holding/default/PV)  \
Instrument/default/Name                                   
Blackrock Equity Dividend Fund                10,836.56   
USD                                             4,323.0   
VANGUARD LIFESTRATEGY GROWTH                  107,900.0   
TOTAL                                        123,059.56   

mndt_xxku9sdf9                  Sum(Holding/default/Units)  
Instrument/default/Name                                     
Blackrock Equity Dividend Fund                       523.0  
USD                                                4,323.0  
VANGUARD LIFESTRATEGY GROWTH                       3,250.0  
TOTAL                                              8,096.0

In [29]:
prettyprint.heading('Mandate', aggregation_results_df[3].columns.name)
aggregation_results_df[3]

Mandate: mndt_uu99sadf2


mndt_uu99sadf2                 Holding/default/SubHoldingKey  \
Instrument/default/Name                                        
Halliburton Co           LusidInstrumentId=LUID_27YIY39Z/USD   
MSCI Inc                 LusidInstrumentId=LUID_OZ0A78EF/USD   
USD                                             Currency=USD   
Under Armour Inc         LusidInstrumentId=LUID_ZLPZOB53/USD   
TOTAL                                                    NaN   

mndt_uu99sadf2           Sum(Holding/default/Cost)  Sum(Holding/default/PV)  \
Instrument/default/Name                                                       
Halliburton Co                             5,268.0                  6,357.0   
MSCI Inc                                  36,347.5                 36,347.5   
USD                                       25,987.0                 25,987.0   
Under Armour Inc                           6,897.0                  7,728.0   
TOTAL                                     74,499.5                 76,419.5   

mndt_uu99sadf2           Sum(Holding/default/Units)  
Instrument/default/Name                              
Halliburton Co                                300.0  
MSCI Inc                                      155.0  
USD                                        25,987.0  
Under Armour Inc                              300.0  
TOTAL                                      26,742.0

![Scopes](img/households-portfoliogroupsmandate.gif)

### Group & aggregate by branch

You can also create a portfolio group to see the investment across all the client accounts across each branch.

*Run the cell below to create a portfolio group for each branch*

In [30]:
# Create the portfolio groups for each mandate
create_portfolio_groups(
    account_data=accounts, 
    grouping_key='branch_id', 
    group_scope=reporting_scope, 
    portfolio_scopes=branch_scopes)

# Aggregate across the portfolio groups
aggregation_results_df = aggregate_portfolio_group(
    grouping_key='branch_id', 
    group_scope=reporting_scope)

Portfolio Group Created
Name: Contains all accounts for the branch_id of Singapore_Singapore
Scope: reporting_38b9-df83-d0e9-c8
Code: Singapore_Singapore-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the branch_id of HongKong_SAR
Scope: reporting_38b9-df83-d0e9-c8
Code: HongKong_SAR-Group
Portfolios Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the branch_id of Singapore_Singapore
Scope: reporting_38b9-df83-d0e9-c8
Code: Singapore_Singapore-Group
Portfolios Inside Group: 
040004-778939522
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the branch_id of Singapore_Singapore
Scope: reporting_38b9-df83-d0e9-c8
Code: Singapore_Singapore-Group
Portfolios Inside Group: 
040004-929987648
040004-778939522
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the branch_id of Singapore_Singapore
Scope: reporting_38b9-df83-d0e9-c8
Code: Singapore_Singapore-Group
Portfolios Inside G

In [31]:
prettyprint.heading('Branch', aggregation_results_df[0].columns.name)
aggregation_results_df[0]

Branch: Singapore_Singapore


Singapore_Singapore                             Holding/default/SubHoldingKey  \
Instrument/default/Name                                                         
Amazon_Nasdaq_AMZN                        LusidInstrumentId=LUID_KWWZD1OC/USD   
Apple_Nasdaq_AAPL                         LusidInstrumentId=LUID_W1UBLAHK/USD   
Blackrock Equity Dividend Fund            LusidInstrumentId=LUID_1L72D4E9/USD   
Brown-Forman Corp                         LusidInstrumentId=LUID_M3VJB8JN/USD   
CAMPBELL SOUP CO                          LusidInstrumentId=LUID_NT3GZV5U/USD   
Halliburton Co                            LusidInstrumentId=LUID_27YIY39Z/USD   
Norwegian Cruise Line Holdings Ltd        LusidInstrumentId=LUID_QUTQJ6OO/USD   
Salesforce.com Inc                        LusidInstrumentId=LUID_WW73RZIP/USD   
T. Rowe Price Instl Small-Cap Stock Fund  LusidInstrumentId=LUID_C5RGDBIJ/USD   
USD                                                              Currency=USD   
Under Armour Inc                          LusidInstrumentId=LUID_ZLPZOB53/USD   
VANGUARD LIFESTRATEGY GROWTH              LusidInstrumentId=LUID_UEP0W6G4/USD   
TOTAL                                                                     NaN   

Singapore_Singapore                       Sum(Holding/default/Cost)  \
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN                                        59,596.86   
Apple_Nasdaq_AAPL                                          10,931.4   
Blackrock Equity Dividend Fund                            10,815.64   
Brown-Forman Corp                                          14,965.0   
CAMPBELL SOUP CO                                            6,637.5   
Halliburton Co                                               723.06   
Norwegian Cruise Line Holdings Ltd                          5,518.4   
Salesforce.com Inc                                        24,008.48   
T. Rowe Price Instl Small-Cap Stock Fund                  13,555.28   
USD                                                        46,523.0   
Under Armour Inc                                            6,756.0   
VANGUARD LIFESTRATEGY GROWTH                              106,600.0   
TOTAL                                                    306,630.62   

Singapore_Singapore                       Sum(Holding/default/PV)  \
Instrument/default/Name                                             
Amazon_Nasdaq_AMZN                                       67,801.5   
Apple_Nasdaq_AAPL                                        10,952.4   
Blackrock Equity Dividend Fund                          10,836.56   
Brown-Forman Corp                                        13,525.0   
CAMPBELL SOUP CO                                          6,289.5   
Halliburton Co                                             826.41   
Norwegian Cruise Line Holdings Ltd                        4,190.4   
Salesforce.com Inc                                      24,082.88   
T. Rowe Price Instl Small-Cap Stock Fund                13,449.64   
USD                                                      46,523.0   
Under Armour Inc                                          7,728.0   
VANGUARD LIFESTRATEGY GROWTH                            107,900.0   
TOTAL                                                  314,105.29   

Singapore_Singapore                       Sum(Holding/default/Units)  
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN                                              39.0  
Apple_Nasdaq_AAPL                                               60.0  
Blackrock Equity Dividend Fund                                 523.0  
Brown-Forman Corp                                              250.0  
CAMPBELL SOUP CO                                               150.0  
Halliburton Co                                                  39.0  
Norwegian Cruise Line Holdings Ltd                              80.0  
Salesforce.com Inc                         

In [32]:
prettyprint.heading('Branch', aggregation_results_df[1].columns.name)
aggregation_results_df[1]

Branch: HongKong_SAR


HongKong_SAR                   Holding/default/SubHoldingKey  \
Instrument/default/Name                                        
Halliburton Co           LusidInstrumentId=LUID_27YIY39Z/USD   
MSCI Inc                 LusidInstrumentId=LUID_OZ0A78EF/USD   
USD                                             Currency=USD   
Under Armour Inc         LusidInstrumentId=LUID_ZLPZOB53/USD   
TOTAL                                                    NaN   

HongKong_SAR             Sum(Holding/default/Cost)  Sum(Holding/default/PV)  \
Instrument/default/Name                                                       
Halliburton Co                             5,268.0                  6,357.0   
MSCI Inc                                  36,347.5                 36,347.5   
USD                                       25,987.0                 25,987.0   
Under Armour Inc                           6,897.0                  7,728.0   
TOTAL                                     74,499.5                 76,419.5   

HongKong_SAR             Sum(Holding/default/Units)  
Instrument/default/Name                              
Halliburton Co                                300.0  
MSCI Inc                                      155.0  
USD                                        25,987.0  
Under Armour Inc                              300.0  
TOTAL                                      26,742.0

![Scopes](img/households-portfoliogroupsscopes.gif)

Finally, you can also create a portfolio group to see the investment across all the client accounts across all branches.

In this case instead of using the functions you defined earlier you will add the group for each branch as sub-groups to the all accounts group.

*Run the cell below to create a portfolio group for all accounts*

In [33]:
# Build a create group request for this group
group_request = models.CreatePortfolioGroupRequest(
    code='allaccounts-Group',
    display_name='Contains all accounts for all branches',
    created=datetime.now(pytz.UTC) - timedelta(days=5000))

# Call LUSID to create the portfolio group
response = api_factory.build(lusid.api.PortfolioGroupsApi).create_portfolio_group(
    scope=reporting_scope,
    create_portfolio_group_request=group_request)

# Pretty print the response
prettyprint.portfolio_group_response(response, 'created')

Portfolio Group Created
Name: Contains all accounts for all branches
Scope: reporting_38b9-df83-d0e9-c8
Code: allaccounts-Group
Portfolios Inside Group: 




In [34]:
 # Iterate over all the scopes
for branch in accounts['branch_id'].unique():

    # Call LUSID to add the sub-group to the all accounts group
    response = api_factory.build(lusid.api.PortfolioGroupsApi).add_sub_group_to_group(
        scope=reporting_scope,
        code='allaccounts-Group',
        resource_id=models.ResourceId(
            scope=reporting_scope,
            code='{}-Group'.format(branch)),
        effective_at=datetime.now(pytz.UTC) - timedelta(days=5000))

    # Pretty print the response
    prettyprint.get_portfolio_group_response(response)

Portfolio Group: 
Name: Contains all accounts for all branches
Scope: reporting_38b9-df83-d0e9-c8
Code: allaccounts-Group
Portfolios Inside Group: 
Subgroups Inside Group: 
Singapore_Singapore-Group


Portfolio Group: 
Name: Contains all accounts for all branches
Scope: reporting_38b9-df83-d0e9-c8
Code: allaccounts-Group
Portfolios Inside Group: 
Subgroups Inside Group: 
HongKong_SAR-Group
Singapore_Singapore-Group




In [35]:
# Create an inline recipe to configue the aggregation logic
inline_recipe = models.ConfigurationRecipe(
    scope="User",
    code='quotes_recipe',
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.LusidInstrumentId.*',
               supplier='DataScope',
               data_scope=reporting_scope,
               quote_type='Price',
               field='Mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='DataScope',
            credit='DataScope',
            equity='DataScope',
            fx='DataScope',
            rates='DataScope'),
        options=models.MarketOptions(
            default_supplier='DataScope',
            default_instrument_code_type='LusidInstrumentId',
            default_scope=reporting_scope)
        )
    )

# Create the aggregation request
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=holdings_effective_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/SubHoldingKey',
        op='Value'),
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/Units',
        op='Sum'),
        models.AggregateSpec(key='Instrument/default/Name',
        op='Value')
    ],
    group_by=[
        'Instrument/default/Name'
    ], 
    portfolio_identifier_code='GroupPortfolio') 

# Call LUSID to perform an aggregation
response = api_factory.build(lusid.api.AggregationApi).get_aggregation(
    scope=reporting_scope,
    code='allaccounts-Group',
    aggregation_request=aggregation_request)

aggregation_results_df = prettyprint.aggregation_response_generic_df(
    response=response, 
    index_key="Instrument/default/Name", 
    name='All Accounts')

In [36]:
prettyprint.heading('All Accounts', '')
aggregation_results_df

All Accounts: 


All Accounts                                    Holding/default/SubHoldingKey  \
Instrument/default/Name                                                         
Amazon_Nasdaq_AMZN                        LusidInstrumentId=LUID_KWWZD1OC/USD   
Apple_Nasdaq_AAPL                         LusidInstrumentId=LUID_W1UBLAHK/USD   
Blackrock Equity Dividend Fund            LusidInstrumentId=LUID_1L72D4E9/USD   
Brown-Forman Corp                         LusidInstrumentId=LUID_M3VJB8JN/USD   
CAMPBELL SOUP CO                          LusidInstrumentId=LUID_NT3GZV5U/USD   
Halliburton Co                            LusidInstrumentId=LUID_27YIY39Z/USD   
MSCI Inc                                  LusidInstrumentId=LUID_OZ0A78EF/USD   
Norwegian Cruise Line Holdings Ltd        LusidInstrumentId=LUID_QUTQJ6OO/USD   
Salesforce.com Inc                        LusidInstrumentId=LUID_WW73RZIP/USD   
T. Rowe Price Instl Small-Cap Stock Fund  LusidInstrumentId=LUID_C5RGDBIJ/USD   
USD                                                              Currency=USD   
Under Armour Inc                          LusidInstrumentId=LUID_ZLPZOB53/USD   
VANGUARD LIFESTRATEGY GROWTH              LusidInstrumentId=LUID_UEP0W6G4/USD   
TOTAL                                                                     NaN   

All Accounts                              Sum(Holding/default/Cost)  \
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN                                        59,596.86   
Apple_Nasdaq_AAPL                                          10,931.4   
Blackrock Equity Dividend Fund                            10,815.64   
Brown-Forman Corp                                          14,965.0   
CAMPBELL SOUP CO                                            6,637.5   
Halliburton Co                                             5,991.06   
MSCI Inc                                                   36,347.5   
Norwegian Cruise Line Holdings Ltd                          5,518.4   
Salesforce.com Inc                                        24,008.48   
T. Rowe Price Instl Small-Cap Stock Fund                  13,555.28   
USD                                                        72,510.0   
Under Armour Inc                                           13,653.0   
VANGUARD LIFESTRATEGY GROWTH                              106,600.0   
TOTAL                                                    381,130.12   

All Accounts                              Sum(Holding/default/PV)  \
Instrument/default/Name                                             
Amazon_Nasdaq_AMZN                                       67,801.5   
Apple_Nasdaq_AAPL                                        10,952.4   
Blackrock Equity Dividend Fund                          10,836.56   
Brown-Forman Corp                                        13,525.0   
CAMPBELL SOUP CO                                          6,289.5   
Halliburton Co                                           7,183.41   
MSCI Inc                                                 36,347.5   
Norwegian Cruise Line Holdings Ltd                        4,190.4   
Salesforce.com Inc                                      24,082.88   
T. Rowe Price Instl Small-Cap Stock Fund                13,449.64   
USD                                                      72,510.0   
Under Armour Inc                                         15,456.0   
VANGUARD LIFESTRATEGY GROWTH                            107,900.0   
TOTAL                                                  390,524.79   

All Accounts                              Sum(Holding/default/Units)  
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN                                              39.0  
Apple_Nasdaq_AAPL                                               60.0  
Blackrock Equity Dividend Fund                                 523.0  
Brown-Forman Corp                                              250.0  
CAMPBELL SOUP CO                   

![Scopes](img/households-groupsall.gif)